In [1]:
import os
import sys
import pandas
import hashlib
import hmac
import requests
import datetime
import time
import json
from tqdm import tqdm
import configparser

In [2]:
# APIアカウント
config = configparser.ConfigParser()
config.read('./config.ini')

BF_ACCESSKEY = config['BITFLYER']['ACCESSKEY']
BF_SECRETKEY = config['BITFLYER']['SECRETKEY']
CC_ACCESSKEY = config['COINCHECK']['ACCESSKEY']
CC_SECRETKEY = config['COINCHECK']['SECRETKEY']

In [3]:
def bf_api(method, path, param):
    base_url = "https://api.bitflyer.com"
    timestamp = str(datetime.datetime.today())
    body = json.dumps(param)
    message = timestamp + method + path + body if (param is not None) else timestamp + method + path
    signature = hmac.new(bytearray(BF_SECRETKEY.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()
    headers = {
        'ACCESS-KEY' : BF_ACCESSKEY,
        'ACCESS-TIMESTAMP' : timestamp,
        'ACCESS-SIGN' : signature,
        'Content-Type' : 'application/json'
    }
    if method == 'POST':
        return requests.post( base_url + path , data = body , headers = headers)
    elif method == 'GET':
        return requests.get( base_url + path , headers = headers)
    
def bf_getBalance():
    return bf_api("GET", "/v1/me/getcollateral", None)
    
def bf_getPositions():
    return bf_api("GET", "/v1/me/getpositions?product_code=FX_BTC_JPY", None)

In [11]:
class Coincheck:
    def __init__(self, access_key, secret_key, url='https://coincheck.com'):
        self.access_key = access_key
        self.secret_key = secret_key
        self.url = url

    def get(self, path, params=None):
        if params != None:
            params = json.dumps(params)
        else:
            params = ''
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
        signature = self.getSignature(message)
        return requests.get(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def post(self, path, params):
        params = json.dumps(params)
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
        signature = self.getSignature(message)
        return requests.post(
            self.url+path,
            data=params,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def delete(self, path):
        nonce = str(int(time.time()))
        message = nonce + self.url + path
        signature = self.getSignature(message)
        return requests.delete(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def getSignature(self, message):
        signature = hmac.new(
            bytes(self.secret_key.encode('ascii')),
            bytes(message.encode('ascii')),
            hashlib.sha256
        ).hexdigest()

        return signature

    def getHeader(self, access_key, nonce, signature):
        headers = {
            'ACCESS-KEY': access_key,
            'ACCESS-NONCE': nonce,
            'ACCESS-SIGNATURE': signature,
            'Content-Type': 'application/json'
        }
        return headers
    

In [73]:
json.loads(bf_getBalance().text)

{'collateral': 99985.0,
 'open_position_pnl': 0.0,
 'require_collateral': 0.0,
 'keep_rate': 0.0}

In [13]:
coincheck = Coincheck(CC_ACCESSKEY, CC_SECRETKEY)
coincheck.get("/api/accounts/balance")

{'success': True,
 'jpy': '200000.0',
 'btc': '0.0',
 'eth': '0',
 'etc': '0',
 'lsk': '0',
 'fct': '0',
 'xrp': '0.0',
 'xem': '0.0',
 'ltc': '0',
 'bch': '0',
 'mona': '0',
 'xlm': '0',
 'qtum': '0',
 'bat': '0',
 'jpy_reserved': '0',
 'btc_reserved': '0',
 'eth_reserved': '0',
 'etc_reserved': '0',
 'lsk_reserved': '0',
 'fct_reserved': '0',
 'xrp_reserved': '0',
 'xem_reserved': '0',
 'ltc_reserved': '0',
 'bch_reserved': '0',
 'mona_reserved': '0',
 'xlm_reserved': '0',
 'qtum_reserved': '0',
 'bat_reserved': '0',
 'jpy_lend_in_use': '0.0',
 'btc_lend_in_use': '0.0',
 'eth_lend_in_use': '0.0',
 'etc_lend_in_use': '0.0',
 'lsk_lend_in_use': '0.0',
 'fct_lend_in_use': '0.0',
 'xrp_lend_in_use': '0.0',
 'xem_lend_in_use': '0.0',
 'ltc_lend_in_use': '0.0',
 'bch_lend_in_use': '0.0',
 'mona_lend_in_use': '0.0',
 'xlm_lend_in_use': '0.0',
 'qtum_lend_in_use': '0.0',
 'bat_lend_in_use': '0.0',
 'jpy_lent': '0.0',
 'btc_lent': '0.0',
 'eth_lent': '0.0',
 'etc_lent': '0.0',
 'lsk_lent': '0